Time to hit the beach!

To stop people going straight to the best sunbeds, the hotel has arranged a labyrinthine way of getting to the beach. Instead of heading straight to your chosen sunbed, you pick a lane and follow it through the maze. When you get to a cross-path, you walk along it to the other lane, then carry on down to the beach.

This example network has six lines, numbered zero to five, and fifteen links. You start at the top and move down.

￼![Example labyrinth](small-expanded-trace.svg.png)

The dashed coloured lines show you some paths you would take going through this labyrinth to the beach. For instance, if you started in lane 1 (the red line), you would switch to lane 4 then lane 3, and you'd emerge on sunbed 3. Entering on lane 5 (the green line) would immediately switch to lane 2, then lane 0, then then lane 4, then finally back to lane 2 where they emerge. Entering on lane zero would take you all round the houses, including crossing the previous tracks, to finally end up back on lane 0.

If you and five friends labelled yourselves `a`, `b`, `c`, `d`, `e`, `f`, your labels when you came out would be in order acfbed.

You'd rather like to know where you and your friends are ending up on the beach, so you've got a copy of the layout plan of the labyrinth. It's given as a sequence of pairs, showing the lane to and from for each cross-path. For instance, the labyrinth above would be described as:

```
(2, 5)
(1, 4)
(0, 3)
(0, 3)
(0, 5)
(3, 5)
(0, 2)
(3, 4)
(2, 4)
(1, 2)
(0, 4)
(1, 2)
(2, 4)
(0, 4)
(1, 4)
```

For each pair `(a, b)`, you can assume $0 \le a < b < n$, if there are _n_ lanes. (In other words, all lane numbers are valid, the first lane number is always less than the second, and cross-paths don't go from a lane back to the same lane.)

# Part 1

The full labyrinth description is given in [04-lines.txt](04-lines.txt). The labyrinth has 26 lines, labelled 0 to 25 inclusive. If you and 25 friends, labelled `a` to `z` in order, entered the labyrinth, in what order would you come out?

(Your answer should be one string of 26 letters, without spaces or punctuation, like `acfbed` .)

# Worked solution

> Note: the code in this notebook doesn't really follow the structure of the problem. That's because I was working on this task for a while, looking at different approaches and different challenges to pose based on the idea. I'll pick out the important parts from the code that's below.

> Also note that terminology changes throughout the code here. The question talks about lanes and distances, but the code refers to lines and heights.


## Data structure
This task requires a bit of though behind the data structures before you can start tackling the problem. The first thing to do is think about the data structure to use to represent the network, and the operations we need to perform on it.

The operations are:
1. Create the network from a collection of links.
2. Follow a person through the labyrinth.
3. Follow lots of people together through the layrinth (if that's different and easier).
4. (Looking ahead) Shuffle the heights of links for packing.

We could consider the labyrinth as a collection of links that affect lanes, or as a collection of lanes that know about links. Given that we're presented with a set of links, and need to move the links around, let's go with representing the network as a bunch of links. 

How to represent each link? As we're doing things with links later, it's easier if I just store the labyrinth as a bunch of links, and have each link know everything about itself that I would want. Given that we could, later, have more than one link at each height, each link will need to know its own left end, right end, and height. 

Python doesn't have anything like records from other languages. I could use a `dict` to store each link, but in this case I'll use a `namedtuple` from the `collections` library. It will allow me to say things like `this_link.height` and `this_link.left`, which is easier to read (and write!) than `this_link['height']` and `this_link['left']`.

## Reading the input
Each line of the file consists of two sequences of digits with sequences of non-digits surrounding them. I use a regular expression and the `re.split()` function to split the line, treating non-digits (the `\D+` term) as the separators. The `read_net` procedure reads the file, splits each line into the substrings, converts the relevant parts into numbers, then builds the links. Note the use of the `enumerate` built-in function, which iterates through a sequence, returning both the item and the count each time. That gives me the heights of the links. 

## Following people through
`follow()` follows one person through the labyrinth. The `line` variable holds the line/lane this person is currently on as they move through the laybrinth.

The procedure is structured to allow for there being several links at the same height. It finds the distinct heights, puts them in order, then iterates through the heights. It then finds all the links at that height and, if one of them has an end on `line`, it uses that link to do the swap. 

This is fine for one person, but it's slow to execute the whole process 26 times for 26 people, when most of the work is the same for each person going through. But I've implemented that process to work on packed networks (the part 2 problem), so let's look at that first.

## Packing
The idea of packing is to keep track of the position of the furthest link that's on each lane. When we add a link to the packed network, we look up the lanes it joins, find the furthest link on either of them, then add the new link at one level beyond that. We then update the positions recorded for the two lanes. 

The current lane distances are held in the `line_heights` dictionary. It's a `defaultdict`, defined to return the value of `-1` for any line that hasn't been processed yet. The height for a new link is the maximum existing height for either of its ends, +1. This means that the first link is placed at height zero.

## Following again
Once we have the idea of a packed network, I clarified the idea of a `height_group`, the set of links that are all at a particular height. The `height_groups()` function uses some library magic to split a network into a list of lists of links, with each inner list being all the links at that height, i.e. it returns a list like this:

```
[<list of links at height 0>, <list of links at height 1>, … <list of links at height n>]
```

Once you have the height groups, you can use them to follow many items through the network at the same time. `follow_many()` takes a sequence of things in their starting order, and follows them all through the network. There must be at least as many items in the input as there are lanes in the network, and I don't check for that being true. The input sequence is converted to a `list`, as that can be updated in place, while Python won't allow changes inside `string`s.

As the packing and height-group-finding ensures that there is at most one link on each lane in any particular height group, I don't need to go through the height group in any particular order. I just take each link swap the items at the ends, and update the `seq` accordingly. (Note the simultaneous assignment for swapping without a temporary variable.)

In [1]:
import collections
import string
import itertools
import re

In [2]:
Link = collections.namedtuple('Link', 'height left right')

In [3]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [4]:
def read_net_string(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [23]:
re.split('\D+', '(2, 4)')

['', '2', '4', '']

In [20]:
def read_net(filename, rev=False):
    with open(filename) as f:
        pairs = [re.split('\D+', p.strip()) for p in f]
        if rev:
            lrs = [(int(lr[1]), int(lr[2])) for lr in reversed(pairs)]
        else:
            lrs = [(int(lr[1]), int(lr[2])) for lr in pairs]
        return [Link(h, l, r) 
                for h, (l, r) in enumerate(lrs)]

In [21]:
small_net = read_net('04-small.txt')
small_net

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [22]:
net = read_net('04-lines.txt')
len(net)

10135

In [36]:
permnet = read_net('permutations.txt')
len(permnet)

23

In [41]:
rpermnet = read_net('permutations.txt', rev=True)
len(rpermnet)

23

In [18]:
def show_net(links, pair_sep=', '):
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [19]:
def link_ends(link):
    return set((link.left, link.right))

In [20]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

You've noticed that the beach labyrinth is longer than it needs to be. Rather than putting each cross-link on a separate step away from the hotel, it's possible to put several cross-links at the same distance, so long as none of them shares an end. 

For instance, the sample labyrinth

￼![Example labyrinth](small-expanded.svg.png)

can be packed into this form, with the first three cross-links all placed at the start of the labyrinth. 

￼![Packed labyrinth](small-packed.svg.png)

You can pack a labyrinth by sliding a link up until just before either of its ends would touch an earlier link. The first 2-5 link stays where it is. The next two links (1-4 and 0-3) slide up to also be on the same level. The next 0-3 link then slides up until it's one step from the start. The other links slide up in turn.

This packed labyrinth has the same shuffling behaviour of the original. But where the last cross-link in the original labyrinth is fourteen steps beyond the start, the last cross-link in the packed labyrinth is only ten steps on.

Only slide links; don't be tempted to remove any. The packed labyrinth should have the same number of cross-links as the original.

# Part 2

The labyrinth is still given in 04-lines.txt. 

After all the packing and sliding, how far is the last cross-link from the first?

## Note to self
Several solutions tried a simplistic approach of packing a layer until a link could not go on that layer, then starting the next. But, this didn't allow for links that could slide more than one layer.

The simple algorithm moved from left net to centre net, but missed you could still pack to form right net.

![Packing variants](packing-variants.svg.png)

In [21]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [22]:
max(l.height for l in small_net)

14

In [23]:
max(l.height for l in pack(small_net))

10

In [24]:
max(l.height for l in net)

10134

In [25]:
pnet = pack(net)

In [26]:
max(l.height for l in pnet)

2286

In [27]:
def height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [28]:
def follow_many(in_sequence, net):
    hgs = height_groups(net)
    seq = list(in_sequence)
    for h in hgs:
        for link in hgs[h]:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [29]:
''.join(follow_many('abcdef', small_net))

'acfbed'

In [69]:
for i in range(len(small_net)+1):
    pre_net = small_net[:i]
    if i == 0:
        print('{:2}'.format(i), 
          "      ".format(small_net[i-1].left, small_net[i-1].right),
          follow_many("012345", pre_net))
    else:
        print('{:2}'.format(i), 
          "({}, {})".format(small_net[i-1].left, small_net[i-1].right),
          follow_many("012345", pre_net))

        

 0        ['0', '1', '2', '3', '4', '5']
 1 (2, 5) ['0', '1', '5', '3', '4', '2']
 2 (1, 4) ['0', '4', '5', '3', '1', '2']
 3 (0, 3) ['3', '4', '5', '0', '1', '2']
 4 (0, 3) ['0', '4', '5', '3', '1', '2']
 5 (0, 5) ['2', '4', '5', '3', '1', '0']
 6 (3, 5) ['2', '4', '5', '0', '1', '3']
 7 (0, 2) ['5', '4', '2', '0', '1', '3']
 8 (3, 4) ['5', '4', '2', '1', '0', '3']
 9 (2, 4) ['5', '4', '0', '1', '2', '3']
10 (1, 2) ['5', '0', '4', '1', '2', '3']
11 (0, 4) ['2', '0', '4', '1', '5', '3']
12 (1, 2) ['2', '4', '0', '1', '5', '3']
13 (2, 4) ['2', '4', '5', '1', '0', '3']
14 (0, 4) ['0', '4', '5', '1', '2', '3']
15 (1, 4) ['0', '2', '5', '1', '4', '3']


In [66]:
small_net[:15]

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [30]:
%%timeit
follow_many('abcdefghij', small_net)

10000 loops, best of 3: 42 µs per loop


In [37]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [39]:
''.join(follow_many(string.ascii_lowercase, permnet))

'zfrasxwigvjoembqcyhplnktud'

In [42]:
''.join(follow_many(string.ascii_lowercase, rpermnet))

'doqzmbishkwunvltpcexyjgfra'

In [43]:
follow_many(string.ascii_lowercase, net) == follow_many(string.ascii_lowercase, rpermnet)

True

In [32]:
%%timeit
follow_many(string.ascii_lowercase, net)

10 loops, best of 3: 19.3 ms per loop


In [33]:
%%timeit
follow_many(string.ascii_lowercase, pnet)

100 loops, best of 3: 18.7 ms per loop
